# Modelo de regressão de imóveis previsão de imovel Airbnb RJ

In [ ]:
from platform import python_version

print('Versão do Python neste projeto:', python_version())

In [ ]:
!pip install -q -U watermark

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go

In [ ]:
%reload_ext watermark
%watermark -a "Rafael Gallo" --iversions

In [ ]:
plt.style.use('seaborn-darkgrid')
sns.set_style("darkgrid") 

In [ ]:
df = pd.read_csv("Dados/HousePrices_HalfMil.csv", sep = ";")
df.head()

In [ ]:
df_train = pd.read_csv("Dados 2/calendar.csv")
df_test = pd.read_csv("Dados 2/listings_summary.csv")

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
df_train.tail()

In [ ]:
df_test.tail()

In [ ]:
df_test.columns = ["ID",
             "Nome",
             "hospede_id",
             "hospede_nome",
             "Bairro",
             "Vizinhança",
             "latitude",
             "longitude",
             "Tipo de sala",
             "Preço",
             "Noites mínimas",
             "Número de comentários",
             "Última revisão",
             "Avaliações por mês",
             "Contagem de listagens de host calculada",
             "Disponibilidade 365"]

df_test.head()

# Análise Exploratória

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
df_train.columns

In [ ]:
df_test.columns

In [ ]:
df_train.dtypes

In [ ]:
df_test.dtypes

In [ ]:
df_train.describe()

In [ ]:
df_test.describe()

In [ ]:
df_train_corr = df_train.corr()
df_train_corr

In [ ]:
df_test_corr = df_test.corr()
df_test_corr

In [ ]:
fig = plt.figure(figsize = (12, 9))

sns.heatmap(df_train_corr,  cmap = 'plasma', annot = True);
plt.show()

In [ ]:
fig = plt.figure(figsize = (12, 9))

sns.heatmap(df_test_corr,  cmap = 'plasma', annot= True);
plt.show()

In [ ]:
plt.figure(figsize=(18, 8))

ax = sns.scatterplot(x="Preço", y="Noites mínimas", hue = 'Bairro', data=df_test)
ax.set_title('Condomínio x Preco')
ax.set_ylabel('Condomínio')
ax.set_xlabel('Preco')

In [ ]:
plt.figure(figsize=(10, 8))

sns.lineplot(x = "Noites mínimas", y = "Preço", data = df_test)

In [ ]:
plt.figure(figsize=(10, 8))

sns.lineplot(x = "Avaliações por mês", y = "Preço", data = df_test)

In [ ]:
plt.figure(figsize=(10, 6))

sns.scatterplot(x = "latitude", y = "longitude", data = df_test, hue = "Tipo de sala")

In [ ]:
apt = df_test[["latitude", "longitude", "Preço"]]

fig = px.scatter_mapbox(apt,
                       lat = "latitude",
                       lon = "longitude",
                       size = "Preço",
                       color_continuous_scale = px.colors.cyclical.IceFire, 
                       size_max = 15, 
                       zoom = 10)

fig.update_layout(
        title = 'Mapa região de apartamentos Rio de Janeiro',
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
apt = df_test[["latitude", "longitude", "Disponibilidade 365"]]

fig = px.scatter_mapbox(apt,
                       lat = "latitude",
                       lon = "longitude",
                       size = "Disponibilidade 365",
                       color_continuous_scale = px.colors.cyclical.IceFire, 
                       size_max = 15, 
                       zoom = 10)

fig.update_layout(
        title = 'Mapa região de apartamentos Nova York',
)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# Limpeza de dados

In [ ]:
df_test.info()

In [ ]:
df_test.drop(["ID", "Nome", "latitude", "longitude", "hospede_id", "Última revisão"], axis = 1, inplace = True)
df_test.drop(["hospede_nome"], axis = 1, inplace = True)
df_test.head()

In [ ]:
df_test["Avaliações por mês"] = df_test["Avaliações por mês"].fillna(0)
df_test

In [ ]:
df_test.isna().sum()

In [ ]:
x = df_test[["Preço"]].values.reshape(-1,1)
y = df_test["Preço"].values.reshape(-1,1)

In [ ]:
x

In [ ]:
y

In [ ]:
x.shape

In [ ]:
y.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler_x = scaler.fit_transform(x)
scaler_y = scaler.fit_transform(y)

In [ ]:
scaler_x

In [ ]:
scaler_y

In [ ]:
scaler_x.shape

In [ ]:
scaler_y.shape

# Treino e Teste

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaler_x, scaler_y, test_size = 0.2, random_state = 0)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_test.shape

In [ ]:
y_test.shape

# Modelo de regressão linear

In [ ]:
from sklearn.linear_model import LinearRegression

reg_line = LinearRegression()
reg_line.fit(X_train, y_train)
reg_pred = reg_line.predict(X_train)
reg_pred

In [ ]:
reg_line.score(X_train, y_train)

In [ ]:
reg_line.coef_

In [ ]:
reg_line.coef_ * 27.74456356 + reg_line.intercept_

In [ ]:
pred = reg_line.predict(X_train)
pred

In [ ]:
y_pred = reg_line.predict(X_test)
y_pred

In [ ]:
pred2 = y_train - pred
pred2

In [ ]:
plt.figure(figsize=(18, 8))
plt.scatter(pred, y_train)
plt.plot(pred, reg_line.predict(X_train), color = "red")
plt.title("Grafico de regressão linear - AirBnb RJ", fontsize = 20)
plt.xlabel("Valor")
plt.ylabel("Valor do imóvel")
plt.legend(["Valor", "Imóvel"])

In [ ]:
ax = sns.distplot(pred)
ax.figure.set_size_inches(20, 8)
ax.set_title('Distribuição de Frequências dos Resíduos', fontsize=18)
ax.set_xlabel('Imóvel', fontsize=14)
ax

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn import metrics
from math import sqrt

print('MSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print("RMSE", np.sqrt(mean_squared_error(y_test, y_pred)))
print("R2", r2_score(y_test, y_pred))

# Previsão do imovel

In [ ]:
X_test[0:1]

In [ ]:
a1 = X_test[0: 1]
a1

In [ ]:
pred = reg_line.predict(a1)[0]
pred

# Salvando o modelo de regressão linear

In [ ]:
import pickle

saida = open("modelo_previsao_imóvel_AirBnb-RJ", "wb")
saida.close()